In [ ]:
output_csv_path = "../data/train_images_filtered_no_duplicates.csv"
train_df_filtered.to_csv(output_csv_path, index=False)

print(f"New CSV generated at: {output_csv_path}")

end_time = datetime.datetime.now()
print(end_time - start_time)

In [ ]:
train_df_filtered = train_df
print(train_df_filtered)

In [ ]:
unique_labels = train_df_filtered.label.value_counts()
num_unique_labels = unique_labels.nunique()
print(unique_labels)

In [ ]:
from sklearn import model_selection

df_train, df_temp = model_selection.train_test_split(
    train_df_filtered, test_size=0.4, random_state=109, stratify=train_df_filtered["label"].values
)

df_valid, df_test = model_selection.train_test_split(
    df_temp, test_size=0.5, random_state=109, stratify=df_temp["label"].values
)

In [ ]:
from imblearn.over_sampling import SMOTE

# Set SMOTE to balance all classes to the size of the largest class
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from skimage.io import imread
import os
# Step 1: Load Images
image_paths = "../dataset/train_images/"  

smote_output_folder = "../dataset/smote_images/"   
os.makedirs(smote_output_folder, exist_ok=True)

X = []
y = []

for idx, row in train_df.iterrows():
    image_id = row['image_id']
    label = row['label']
    
    # Load each image, assuming file path format is image_paths + image_id + '.jpg'
    image = imread(f"{image_paths}{image_id}")
    
    # Append image and label
    X.append(image)
    y.append(label)

X = np.array(X)  # Convert to numpy array
y = np.array(y)

In [ ]:

# Step 2: Flatten images if necessary
X_flattened = X.reshape(X.shape[0], -1)  # Flatten to 1D if needed

# Step 3: Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled_flattened, y_resampled = smote.fit_resample(X_flattened, y)

# Step 4: Reshape back if you flattened
image_shape = X.shape[1:]  # Original shape
X_resampled = X_resampled_flattened.reshape(-1, *image_shape)

# Step 5: Create DataFrame with resampled data
# Optionally, create synthetic `image_id`s for new samples or add a prefix to differentiate
df_resampled = pd.DataFrame({
    'image_id': [f"{i}_synthetic" for i in range(len(y_resampled))],  # New synthetic IDs
    'class_label': y_resampled
})

# Save SMOTE-generated images
for i, img_array in enumerate(X_resampled[len(X):]):  # Only new images
    img = Image.fromarray(img_array.astype('uint8'), 'RGB')  # Convert numpy array to Image
    img.save(f"{smote_output_folder}/smote_image_{i}.jpg")  # Save with unique name


In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler

# desired_majority_class_size = 6000

# class_counts = df_train["labels"].value_counts()
# undersample_strategy = {class_counts.idxmax(): desired_majority_class_size}

# rus = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=109)
# X_under, y_under = rus.fit_resample(
#     df_train["image_id"].values.reshape(-1, 1), df_train["labels"].values
# )

# desired_minority_class_size = 6000

# ros = RandomOverSampler(
#     sampling_strategy={
#         label: desired_minority_class_size
#         for label in class_counts.index
#         if class_counts[label] < desired_minority_class_size
#     },
#     random_state=109,
# )
# X_resampled, y_resampled = ros.fit_resample(X_under, y_under)

# df_train_resampled = pd.DataFrame(
#     {"image_id": X_resampled.flatten(), "labels": y_resampled}
# )

# # Check the new class distribution
# print(df_train_resampled["labels"].value_counts())

# df_train_resampled.reset_index(drop=True, inplace=True)


In [ ]:
output_csv_path = "../data/train_images_filtered_no_duplicates.csv"
train_df_filtered.to_csv(output_csv_path, index=False)

print(f"New CSV generated at: {output_csv_path}")

end_time = datetime.datetime.now()
print(end_time - start_time)
